## 문자 단위 RNN(Char RNN)

### 1. 문자 단위 RNN (Char RNN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

#### 1) 훈련 데이터 전처리하기

구현할 것 : ```apple``` $\rightarrow$ ```pple!```

##### 입력 데이터와 레이블 데이터에 대해서 문자 집합(vocabulary)을 만듦

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [3]:
char_vocab

['!', 'a', 'e', 'l', 'p']

In [5]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [21]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [23]:
index_to_char = dict((i, c) for c, i in char_to_index.items()) # 문자에 고유한 정수 인덱스 부여
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)


[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [7]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [10]:
# 원-핫 인코딩
x_one_hot = np.array([np.eye(vocab_size)[x] for x in x_data])
print(x_one_hot)


[[[0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0.]]]


In [11]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [12]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


#### 2) 모델 구현하기

In [13]:
class CharRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [26]:
model = CharRNN(input_size, hidden_size, output_size)

In [ ]:
outputs = model(X)
print(outputs.shape)  # batch_size, sequence_length, output_size

torch.Size([1, 5, 5])


In [27]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)

In [28]:
for i in range(100):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.view(-1, output_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)


0 loss:  1.7317272424697876 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
1 loss:  1.4012726545333862 prediction:  [[4 4 2 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppeee
2 loss:  1.2407243251800537 prediction:  [[4 4 3 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplee
3 loss:  1.074204444885254 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.9040333032608032 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.7173529863357544 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.5446842908859253 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.4152960777282715 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.3083057403564453 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.2193511724472046 prediction:  [[4 4 3 2 0]] t

### 2. 더 많은 데이터로 학습한 문자 단위 RNN (Char RNN)

#### 1) 훈련 데이터 전처리하기

In [29]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")


In [30]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩


In [31]:
print(char_dic) # 공백도 여기서는 하나의 원소


{' ': 0, 'o': 1, 'n': 2, 'm': 3, 'i': 4, 'd': 5, 'f': 6, 'c': 7, 'b': 8, 'p': 9, '.': 10, 'a': 11, 't': 12, 'y': 13, 'g': 14, 'r': 15, 'l': 16, 'e': 17, 's': 18, 'w': 19, ',': 20, 'k': 21, 'u': 22, "'": 23, 'h': 24}


In [32]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))


문자 집합의 크기 : 25


In [33]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1


In [34]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index


0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [35]:
print(x_data[0])    # if you wan
print(y_data[0])    # f you want


[4, 6, 0, 13, 1, 22, 0, 19, 11, 2]
[6, 0, 13, 1, 22, 0, 19, 11, 2, 12]


In [37]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)


In [38]:
X.shape

torch.Size([170, 10, 25])

In [39]:
print(X[0])
print(Y[0])

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

#### 2) 모델 구현하기

In [47]:
class CharRNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(CharRNN, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x


In [48]:
model = CharRNN(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.


In [49]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)


In [50]:
for i in range(100):
    optimizer.zero_grad()
    outputs = model(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)


kkskkskskkkkkkskkkksssssskkkksskkksslskkkkskkskkkskkskksosskksskskkskksksskskksskkkkssskkkksksoskkksskkskkssksskkskkkosksssskkssskkskskkksskkskkksoksssskkskskkkkskkkkkokskkkkossks
  ee       e een  e e   e      ee    ee      eee   ee  e  e ee  ee   ee       e  ee    eee e e ee         e      e  e   e  e    e  e ee  ee ee    t e     e  et  ee  e e     e  e  
o  o  o    to    o  to      t   o o  o  o   o  o           o to o    t     o  t     oo  o  o  o  o o  to  t   o t    o      o o  o  o to o   o o  t    o  t   t   o     o  o  o  o 
o .o i..do  o oo e   o oei  eoeo  eoeo  e e  e e  oe e emeo  io e  e eoeo  e e oeie eo oe eo  o  o i  eoe eoeoe eoe eoe emeo     e eo  o io eoeo i  eodo ieoeiaoe e e o ie eoeoeeei
  e e e e e e me ee e e eme e eme ememe o  ee  e  ee e eme e me  e e eme e oe e eme eme e   e me n  e eme e e e eoe e e eme e  e   eme n  e  e oe e ememe eme e e e ee eme e eme e 
    d ne n do  nd d n  o o   n l  ln l do do  le e  o   l lo  n  d n  o   do  n  l   n l   n  o  o  